In [1]:
import pandas as pd
import plotly.express as px

from src.utils import *
from src.features.base import *
from src.features.volume_obs import *
from src.features.swe import *
from src.data.base import *
from src.config import *
from src.models.postprocess import *
from src.models.lgb import *
from src.models.ensemble import *

import importlib

EXP_NAME = 'swe_k9_s1_dp'
cfg = importlib.import_module(f"configs.forecast.{EXP_NAME}").cfg

val_years=[2020,2021,2022]
test_years=[2023]

# Evaluation

In [2]:
exp_list = [
    "swe_k9_s1_dp",
    "swe_k9_s1_dp_s1024",
    "swe_k9_s1_dp_s3024",
    "swe_k5_s3_dp",
    "swe_k5_s3_dp_s1024",
    "swe_k5_s3_dp_s3024",
    "pdsi_swe_k9_s1_dp",
    "pdsi_swe_k9_s1_dp_s1024",
    "pdsi_swe_k9_s1_dp_s3024",
    "pdsi_swe_k5_s3_dp",
    "pdsi_swe_k5_s3_dp_s1024",
    "pdsi_swe_k5_s3_dp_s3024",
]
groupby_cols=["site_id", "year", "month", "day", "md_id"]

df_pred_val_all = []
df_pred_test_all = []
for exp_name in exp_list:
    df_pred = pd.read_csv(f"runs/forecast_v2_prd/{exp_name}/pred.csv").query('(site_id=="detroit_lake_inflow" & md_id>=24)==False')
    df_pred['pred_volume_50'] = df_pred['pred_volume_reg']

    df_pred_val = df_pred[df_pred["year"].isin(val_years)]
    df_pred_test = df_pred[df_pred["year"].isin(test_years)]


    df_pred_val_all.append(df_pred_val.assign(model_id=exp_name.replace("_s1024", "").replace("_s3024", "")))
    df_pred_test_all.append(df_pred_test.assign(model_id=exp_name.replace("_s1024", "").replace("_s3024", "")))

df_pred_val_all = pd.concat(df_pred_val_all)
df_pred_test_all = pd.concat(df_pred_test_all)
df_pred_val_all = mean_ensemble(df_pred_val_all, groupby_cols=groupby_cols + ["model_id"])
df_pred_test_all = mean_ensemble(df_pred_test_all, groupby_cols=groupby_cols + ["model_id"])
df_pred_val_all = clip_prediction(rearrange_prediction(df_pred_val_all))
df_pred_test_all = clip_prediction(rearrange_prediction(df_pred_test_all))

df_pred_val_ens = df_pred_val_all.copy()
df_pred_test_ens = df_pred_test_all.copy()

df_test = read_train(is_forecast=True)
df_test = df_test[df_test["year"].isin(cfg["test_years"])]
df_pred_test_ens = pd.merge(
    df_pred_test_ens.drop(columns=["volume"]),
    df_test
)

df_pred_val_ens["cat"] = "val"
df_pred_test_ens["cat"] = "test"

df_pred_val_ens_pp = use_previous_forecast_sites(df_pred_val_ens,
                                                 months=[5,6,7],
                                                 cols=["pred_volume_10", "pred_volume_50"])
df_pred_test_ens_pp = use_previous_forecast_sites(df_pred_test_ens,
                                                  months=[5,6,7],
                                                  cols=["pred_volume_10", "pred_volume_50"])

In [3]:
eval_all(df_pred_val_ens, [["model_id"]], is_include_mean_std=False)
eval_all(df_pred_test_ens, [["model_id"]], is_include_mean_std=False)

['model_id']


,n,mpl,mpl10,mpl50,mpl90,int_cvr,rmse,r2,mape,bias,actual_mean,pred_mean
model_id,,,,,,,,,,,,
pdsi_swe_k5_s3_dp,2172.0,67.965181,44.167258,108.145063,51.583222,0.604972,200.514673,0.971047,27.288426,-26.023495,753.240214,779.263708
pdsi_swe_k9_s1_dp,2172.0,69.934424,44.457527,110.110362,55.235383,0.616483,207.473100,0.969003,27.238350,-23.049560,753.240214,776.289774
swe_k5_s3_dp,2172.0,70.071194,43.182270,111.465074,55.566237,0.692449,206.313374,0.969348,29.289973,-46.057928,753.240214,799.298142
swe_k9_s1_dp,2172.0,71.561591,43.279247,114.673872,56.731656,0.680479,213.218936,0.967262,29.382998,-45.778875,753.240214,799.019088




['model_id']


,n,mpl,mpl10,mpl50,mpl90,int_cvr,rmse,r2,mape,bias,actual_mean,pred_mean
model_id,,,,,,,,,,,,
pdsi_swe_k5_s3_dp,724.0,82.284300,54.095729,126.564169,66.193003,0.683702,230.469612,0.954654,12.320575,38.758255,1039.918597,1001.160341
pdsi_swe_k9_s1_dp,724.0,81.555323,52.655327,128.807001,63.203641,0.708564,232.468767,0.953864,12.181876,43.640709,1039.918597,996.277888
swe_k5_s3_dp,724.0,83.790833,55.952843,133.431044,61.988612,0.770718,238.890908,0.951280,12.793484,37.058326,1039.918597,1002.860271
swe_k9_s1_dp,724.0,85.801128,55.596925,137.666034,64.140425,0.780387,246.828663,0.947988,12.664245,41.557670,1039.918597,998.360927


In [4]:
eval_all(df_pred_val_ens_pp, [["model_id"]], is_include_mean_std=False)
eval_all(df_pred_test_ens_pp, [["model_id"]], is_include_mean_std=False)

['model_id']


,n,mpl,mpl10,mpl50,mpl90,int_cvr,rmse,r2,mape,bias,actual_mean,pred_mean
model_id,,,,,,,,,,,,
pdsi_swe_k5_s3_dp,2172.0,67.808958,43.306677,108.536974,51.583222,0.603131,199.905680,0.971223,27.664910,-29.967544,753.240214,783.207758
pdsi_swe_k9_s1_dp,2172.0,69.430240,43.581951,109.473385,55.235383,0.610037,204.488063,0.969888,27.550897,-27.316280,753.240214,780.556494
swe_k5_s3_dp,2172.0,70.195412,43.243114,111.776883,55.566237,0.685543,207.592712,0.968967,29.533295,-48.231148,753.240214,801.471361
swe_k9_s1_dp,2172.0,71.636065,43.716983,114.459555,56.731656,0.672652,213.321486,0.967231,29.626818,-47.743288,753.240214,800.983501




['model_id']


,n,mpl,mpl10,mpl50,mpl90,int_cvr,rmse,r2,mape,bias,actual_mean,pred_mean
model_id,,,,,,,,,,,,
pdsi_swe_k5_s3_dp,724.0,81.019615,51.969153,124.896690,66.193003,0.687845,224.107485,0.957123,12.357255,33.743970,1039.918597,1006.174627
pdsi_swe_k9_s1_dp,724.0,79.492526,49.681506,125.592432,63.203641,0.712707,220.549499,0.958474,12.189232,37.401417,1039.918597,1002.517180
swe_k5_s3_dp,724.0,84.038137,55.132690,134.993108,61.988612,0.767956,240.765337,0.950512,12.809144,35.437288,1039.918597,1004.481309
swe_k9_s1_dp,724.0,84.596957,53.382233,136.268213,64.140425,0.777624,243.640936,0.949323,12.601036,37.760399,1039.918597,1002.158198
